# Karger algorithm

## Support objects

In [106]:
class Node:
    def __init__(self,  isContract, tag = -1,):
        self.tag = tag
        self.adjacencyList = []
        self.isContract = isContract

class Graph:
    def __init__(self):
        self.n_nodes, self.n_edges = 0, 0
        self.nodes = defaultdict(Node)

    def createNodes(self, nums: int):
        for i in range(1, nums+1):
            self.nodes[i] = Node(False, i)
            self.n_nodes += 1

    def addNode(self, tag:int, adjTag:int):
        self.nodes[tag].adjacencyList.append(self.nodes[adjTag])
        #self.nodes[adjTag].adjacencyList.append(self.nodes[tag]) # Graph is undirected
        if tag <= adjTag: self.n_edges += 1

    def buildGraph(self):
        self.createNodes(len(dataset))
        for v in dataset:
            for i in range (1, len(v)):
                self.addNode(v[0], v[i])

## Reading the input

In [107]:
from collections import defaultdict
def readInput(path: str):
    lines = open(path, "r").read().split("\n")
    input = []
    for line in range(len(lines)-1):
        input.append(list(map(int, lines[line].split()))) # Transform list of strings to list of integers

    return input

def readOutput(path: str):
    return open(path, "r").read().split("\n")[0]

## Karger implementation

In [108]:
import random
import math
import copy

def makeCopy(g: Graph):
        graph = Graph()
        graph.nodes = copy.deepcopy(g.nodes)
        graph.n_edges, graph.n_nodes = g.n_edges, g.n_nodes
        return graph

def fullContraction(g):
    graph = makeCopy(g)
    for i in range(graph.n_nodes-2):
        print("iteration =", i)
        print("n_nodes =", graph.n_nodes)
        print("n_edges =", graph.n_edges)

        # Select two linked nodes (arch)
        print("len(nodes) =", len(graph.nodes))
        n1 = random.choice([n for n in graph.nodes.values() if not n.isContract])
        n1.isContract = True

        print("selected node = ", n1.tag)
        n2 = random.choice([n for n in n1.adjacencyList if not n.isContract])
        n2.isContract = True

        print(n1.tag,n2.tag, "\n")

        # Contraction
        id = "Z_"+str(n1.tag)+str(n2.tag)
        graph.nodes[id] = Node(False)
        available_nodes = [n for n in graph.nodes.values() if not n.isContract and n != n1 and n != n2]
        
        for n in available_nodes:
            if n1 in n.adjacencyList or n2 in n.adjacencyList:
                n.adjacencyList.append(graph.nodes[id])
                graph.nodes[id].adjacencyList.append(n)
        
        graph.n_nodes -= 1
        graph.n_edges -= 1  # Need to compute molteplicity

        del n1
        del n2

    return graph.n_edges, graph

def karger(graph, k):
    min = math.inf
    for i in range(k):
        print("Round {} out of {}\n".format(i+1, k))
        t, g = fullContraction(graph)
        if t < min: min = t
        print("- "*10, "\n")
    return min, g

In [109]:
dataset = readInput("mincut_dataset/input_random_1_6.txt")
expected_result = readOutput("mincut_dataset/output_random_1_6.txt")
k = 10
graph = Graph()
graph.buildGraph()

#print(graph.n_nodes, graph.n_edges)
result, g = karger(graph, k)
print("expected result = ", expected_result)
print("final result = ", result)
print()
for n in g.nodes.items():
    print(n)

Round 1 out of 10

iteration = 0
n_nodes = 6
n_edges = 9
len(nodes) = 6
selected node =  2
2 4 

iteration = 1
n_nodes = 4
n_edges = 8
len(nodes) = 7
selected node =  5
5 -1 

iteration = 2
n_nodes = 2
n_edges = 7
len(nodes) = 8
selected node =  1
1 -1 

iteration = 3
n_nodes = 0
n_edges = 6
len(nodes) = 9
selected node =  6
6 -1 

- - - - - - - - - -  

Round 2 out of 10

iteration = 0
n_nodes = 6
n_edges = 9
len(nodes) = 6
selected node =  5
5 6 

iteration = 1
n_nodes = 4
n_edges = 8
len(nodes) = 7
selected node =  -1
-1 2 

iteration = 2
n_nodes = 2
n_edges = 7
len(nodes) = 8
selected node =  -1
-1 4 

iteration = 3
n_nodes = 0
n_edges = 6
len(nodes) = 9
selected node =  -1
-1 3 

- - - - - - - - - -  

Round 3 out of 10

iteration = 0
n_nodes = 6
n_edges = 9
len(nodes) = 6
selected node =  4
4 1 

iteration = 1
n_nodes = 4
n_edges = 8
len(nodes) = 7
selected node =  5
5 -1 

iteration = 2
n_nodes = 2
n_edges = 7
len(nodes) = 8
selected node =  -1
-1 6 

iteration = 3
n_nodes = 0
n

In [110]:
#files = os.listdir("mincut_dataset")
#for f in files:
#
#    dataset = readInput("mincut_dataset/" + f)
#    k = 5
#    graph = Graph()
#    graph.buildGraph()
#
#    print()    
#    print("Executing karger on {} dataset file with k = {}".format(f, k))
#    print()
#
#    #print(graph.n_nodes, graph.n_edges)
#    result, g = karger(graph, k)
#    print("final result =", result)
#    for n in g.nodes.items():
#        print(n)